In [1]:
import pyconll #pip3 install this if you don't have it
import torchtext.data as tt
import torchtext
import torch 
import torch.nn as nn
import torch.optim as optim
import time

In [2]:
af_vec = torchtext.vocab.FastText(language='af')
du_vec = torchtext.vocab.FastText(language='nl')

In [3]:
embedding_dim = 300

In [4]:
AFRIKAANS_TRAIN = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
AFRIKAANS_DEV = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-dev.conllu'
AFRIKAANS_TEST = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-test.conllu'

DUTCH_TRAIN = "UD_Dutch-Alpino/nl_alpino-ud-train.conllu"
DUTCH_DEV = "UD_Dutch-Alpino/nl_alpino-ud-dev.conllu"
DUTCH_TEST = "UD_Dutch-Alpino/nl_alpino-ud-test.conllu"

In [5]:
# from https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb
def make_sentences(path):
    data = pyconll.load_from_file(path)
    sentences = []
    tagged_sentences = []
    for each in data:
        tagged_sentence=[]
        sentence = []
        for token in each:
            if token.upos and token.form:
                tagged_sentence.append(token.upos)
                sentence.append(token.form.lower())
        tagged_sentences.append(tagged_sentence)
        sentences.append(sentence)
    return sentences, tagged_sentences

In [6]:
train_afr_raw, tagged_train_afr_raw = make_sentences(AFRIKAANS_TRAIN)
dev_afr_raw, tagged_dev_afr_raw = make_sentences(AFRIKAANS_DEV)
test_afr_raw, tagged_test_afr_raw = make_sentences(AFRIKAANS_TEST)

train_du_raw, tagged_train_du_raw = make_sentences(DUTCH_TRAIN)
dev_du_raw, tagged_dev_du_raw = make_sentences(DUTCH_DEV)
test_du_raw, tagged_test_du_raw = make_sentences(DUTCH_TEST)

In [7]:
print("AFRIKAANS")
print("Tagged sentences in train set: ", len(tagged_train_afr_raw))
print("Tagged words in train set:", len([item for sublist in tagged_train_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_afr_raw))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_afr_raw))
print("Tagged words in test set:", len([item for sublist in tagged_test_afr_raw for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_afr_raw)+len(tagged_dev_afr_raw)+len(tagged_dev_afr_raw))

AFRIKAANS
Tagged sentences in train set:  1315
Tagged words in train set: 33894
Tagged sentences in dev set:  194
Tagged words in dev set: 5317
Tagged sentences in test set:  425
Tagged words in test set: 10065
****************************************
Total sentences in dataset: 1703


In [8]:
print("DUTCH")
print("Tagged sentences in train set: ", len(tagged_train_du_raw))
print("Tagged words in train set:", len([item for sublist in tagged_train_du_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_du_raw))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_du_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_du_raw))
print("Tagged words in test set:", len([item for sublist in tagged_test_du_raw for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_du_raw)+len(tagged_dev_du_raw)+len(tagged_dev_du_raw))

DUTCH
Tagged sentences in train set:  12264
Tagged words in train set: 185999
Tagged sentences in dev set:  718
Tagged words in dev set: 11549
Tagged sentences in test set:  596
Tagged words in test set: 11053
****************************************
Total sentences in dataset: 13700


In [9]:
# from https://github.com/tringm/POSTagger_Pytorch/blob/master/src/util/nlp.py
def build_tag_field(sentences_tokens):
    token_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('tokens', token_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_tokens]
    torch_dataset = tt.Dataset(examples, fields)
    return token_field
    
def build_text_field(sentences_words):
    text_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('text', text_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_words]
    torch_dataset = tt.Dataset(examples, fields)
    return text_field

In [10]:
#fields, AFR
train_afr = build_text_field(train_afr_raw)
dev_afr = build_text_field(dev_afr_raw)
test_afr = build_text_field(test_afr_raw)
tagged_train_afr = build_tag_field(tagged_train_afr_raw)
tagged_dev_afr = build_tag_field(tagged_dev_afr_raw)
tagged_test_afr = build_tag_field(tagged_test_afr_raw)

fields_train_afr = (("text", train_afr), ("udtags", tagged_train_afr))
examples_train_afr = [tt.Example.fromlist(item, fields_train_afr) for item in zip(train_afr_raw, tagged_train_afr_raw)]
fields_dev_afr = (("text", dev_afr), ("udtags", tagged_dev_afr))
examples_dev_afr = [tt.Example.fromlist(item, fields_dev_afr) for item in zip(dev_afr_raw, tagged_dev_afr_raw)]
fields_test_afr = (("text", test_afr), ("udtags", tagged_test_afr))
examples_test_afr = [tt.Example.fromlist(item, fields_test_afr) for item in zip(test_afr_raw, tagged_test_afr_raw)]

train_data_afr = tt.Dataset(examples_train_afr, fields_train_afr)
valid_data_afr = tt.Dataset(examples_dev_afr, fields_dev_afr)
test_data_afr = tt.Dataset(examples_test_afr, fields_test_afr)

#build vocabs so that they are shared between splits
train_afr.build_vocab(train_data_afr, valid_data_afr, test_data_afr)
dev_afr.vocab = train_afr.vocab
test_afr.vocab = train_afr.vocab
tagged_train_afr.build_vocab(train_data_afr, valid_data_afr, test_data_afr)
tagged_dev_afr.vocab = tagged_train_afr.vocab
tagged_test_afr.vocab = tagged_train_afr.vocab

In [11]:
#fields, DUT
train_du = build_text_field(train_du_raw)
dev_du = build_text_field(dev_du_raw)
test_du = build_text_field(test_du_raw)
tagged_train_du = build_tag_field(tagged_train_du_raw)
tagged_dev_du = build_tag_field(tagged_dev_du_raw)
tagged_test_du = build_tag_field(tagged_test_du_raw)

fields_train_du = (("text", train_du), ("udtags", tagged_train_du))
examples_train_du = [tt.Example.fromlist(item, fields_train_du) for item in zip(train_du_raw, tagged_train_du_raw)]
fields_dev_du = (("text", dev_du), ("udtags", tagged_dev_du))
examples_dev_du = [tt.Example.fromlist(item, fields_dev_du) for item in zip(dev_du_raw, tagged_dev_du_raw)]
fields_test_du = (("text", test_du), ("udtags", tagged_test_du))
examples_test_du = [tt.Example.fromlist(item, fields_test_du) for item in zip(test_du_raw, tagged_test_du_raw)]

train_data_du = tt.Dataset(examples_train_du, fields_train_du)
valid_data_du = tt.Dataset(examples_dev_du, fields_dev_du)
test_data_du = tt.Dataset(examples_test_du, fields_test_du)

#build vocabs so that they are shared between splits
train_du.build_vocab(train_data_du, valid_data_du, test_data_du)
dev_du.vocab = train_du.vocab
test_du.vocab = train_du.vocab
tagged_train_du.build_vocab(train_data_du, valid_data_du, test_data_du)
tagged_dev_du.vocab = tagged_train_du.vocab
tagged_test_du.vocab = tagged_train_du.vocab

In [12]:
# Code adapted from https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76
afr_matrix_len = len(train_afr.vocab.itos)
afr_weights_matrix = torch.zeros((afr_matrix_len, embedding_dim))
words_found = 0
words_missing = 0

for i, word in enumerate(train_afr.vocab.itos):
#     print(i, word)
    try: 
        afr_weights_matrix[i] = af_vec[word]
        words_found += 1
    except KeyError:
#         print("test")
        afr_weights_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim, ))
        words_missing += 1

print("Afrikaans words missing: ", words_missing)

du_matrix_len = len(train_du.vocab.itos)
du_weights_matrix = torch.zeros((du_matrix_len, embedding_dim))
words_found = 0
words_missing = 0

for i, word in enumerate(train_afr.vocab.itos):
#     print(i, word)
    try: 
        du_weights_matrix[i] = du_vec[word]
        words_found += 1
    except KeyError:
#         print("test")
        du_weights_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim, ))
        words_missing += 1

print("dutch words missing: ", words_missing)

Afrikaans words missing:  0
dutch words missing:  0


In [13]:
def create_emb_layer(weights_matrix, pad_idx, non_trainable=False):
    input_dim, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, input_dim, embedding_dim

In [14]:
# from https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
#model
batch_size=128
device = torch.device('cpu')

#needs to be tuple of dataset objects
train_iterator, valid_iterator, test_iterator = tt.BucketIterator.splits(
    (train_data_du, valid_data_du, test_data_du), 
    batch_size = batch_size,
    device = device, sort=False)

In [15]:
# try without dropout first
class BiLSTMTagger(nn.Module):
    #https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        #fully connected layer
        self.fc = nn.Linear((hidden_dim * 2 if bidirectional else hidden_dim), output_dim)
     
    
    def forward(self, text):
        embedded = self.embedding(text)
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(outputs)
        return predictions

class BiLSTMTagger_Pretrained(nn.Module):
    #https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
    def __init__(self, weights_matrix, hidden_dim, output_dim, n_layers, bidirectional, pad_idx):
        super().__init__()
        self.embedding, input_dim, embedding_dim = create_emb_layer(weights_matrix, pad_idx, False)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        #fully connected layer
        self.fc = nn.Linear((hidden_dim * 2 if bidirectional else hidden_dim), output_dim)
     
    
    def forward(self, text):
        embedded = self.embedding(text)
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(outputs)
        return predictions

In [16]:
in_dim = len(train_du.vocab)
emb_dim = 100
hid_dim = 128
out_dim = len(tagged_train_du.vocab)
n_layers = 2
bidirectional = True
pad_index = train_du.vocab.stoi[train_du.pad_token]
tag_pad_idx = tagged_train_du.vocab.stoi[tagged_train_du.pad_token]

In [17]:
model = BiLSTMTagger_Pretrained(du_weights_matrix, hid_dim, out_dim, n_layers, bidirectional, pad_index)
criterion = nn.CrossEntropyLoss(ignore_index = tag_pad_idx)
optimizer = optim.Adam(model.parameters())

In [18]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [19]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        text = batch.text
        tags = batch.udtags
        
        optimizer.zero_grad()       
        predictions = model(text)        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        loss = criterion(predictions, tags) 
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
N_EPOCHS = 10

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, tag_pad_idx)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, tag_pad_idx)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 11s
	Train Loss: 1.612 | Train Acc: 50.95%
	 Val. Loss: 0.706 |  Val. Acc: 79.59%
Epoch: 02 | Epoch Time: 1m 53s
	Train Loss: 0.329 | Train Acc: 91.16%
	 Val. Loss: 0.321 |  Val. Acc: 90.34%
Epoch: 03 | Epoch Time: 1m 41s
	Train Loss: 0.130 | Train Acc: 96.46%
	 Val. Loss: 0.269 |  Val. Acc: 91.37%
Epoch: 04 | Epoch Time: 1m 25s
	Train Loss: 0.087 | Train Acc: 97.46%
	 Val. Loss: 0.284 |  Val. Acc: 91.07%
Epoch: 05 | Epoch Time: 1m 16s
	Train Loss: 0.067 | Train Acc: 98.03%
	 Val. Loss: 0.327 |  Val. Acc: 90.88%
Epoch: 06 | Epoch Time: 1m 22s
	Train Loss: 0.053 | Train Acc: 98.43%
	 Val. Loss: 0.312 |  Val. Acc: 90.92%
Epoch: 07 | Epoch Time: 1m 13s
	Train Loss: 0.041 | Train Acc: 98.81%
	 Val. Loss: 0.339 |  Val. Acc: 90.89%
Epoch: 08 | Epoch Time: 1m 11s
	Train Loss: 0.031 | Train Acc: 99.16%
	 Val. Loss: 0.334 |  Val. Acc: 91.09%
Epoch: 09 | Epoch Time: 1m 21s
	Train Loss: 0.024 | Train Acc: 99.39%
	 Val. Loss: 0.406 |  Val. Acc: 91.02%
Epoch: 10 | Epoch T

In [23]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, tag_pad_idx)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.415 |  Test Acc: 91.23%


In [24]:
print(list(model.named_parameters()))

[('embedding.weight', Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1028, -0.0077,  0.1081,  ..., -0.0456, -0.0399,  0.0344],
        ...,
        [ 0.0195,  0.0151, -0.0267,  ..., -0.0334, -0.0371,  0.0006],
        [-0.0266, -0.0268,  0.0068,  ..., -0.0028,  0.0209,  0.0052],
        [ 0.0408, -0.0402,  0.0251,  ..., -0.0093,  0.0122, -0.0263]],
       requires_grad=True)), ('lstm.weight_ih_l0', Parameter containing:
tensor([[ 0.1289,  0.0157, -0.0014,  ..., -0.0678, -0.0357,  0.0552],
        [ 0.0866, -0.1017, -0.0422,  ..., -0.0014, -0.0085, -0.0364],
        [ 0.1485, -0.0080, -0.1188,  ...,  0.0073,  0.0518,  0.0063],
        ...,
        [-0.1028,  0.1237,  0.0354,  ..., -0.0605, -0.0718,  0.1619],
        [ 0.0479, -0.0212, -0.0422,  ...,  0.0163,  0.0509, -0.0606],
        [-0.2298, -0.1074,  0.0397,  ..., -0.0796,  0.1078,  0.1057]],
       requires

In [25]:
#TRANSFER
train_iterator, valid_iterator, test_iterator = tt.BucketIterator.splits(
    (train_data_afr, valid_data_afr, test_data_afr), 
    batch_size = batch_size,
    device = device, sort=False)

In [26]:
in_dim = len(train_afr.vocab)
emb_dim = 100
hid_dim = 128
out_dim = len(tagged_train_afr.vocab)
n_layers = 2
bidirectional = True
pad_index = train_afr.vocab.stoi[train_afr.pad_token]
tag_pad_idx = tagged_train_afr.vocab.stoi[tagged_train_afr.pad_token]

In [27]:
model2 = BiLSTMTagger_Pretrained(afr_weights_matrix, hid_dim, out_dim, n_layers, bidirectional, pad_index)
criterion = nn.CrossEntropyLoss(ignore_index = tag_pad_idx)
optimizer = optim.Adam(model2.parameters())

In [28]:
#populate dutch params in dict
transfer_param_dict = {}
params = model.named_parameters()
for name, param in params:
    transfer_param_dict[name] = param.data

In [29]:
print(transfer_param_dict.keys())

dict_keys(['embedding.weight', 'lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0', 'lstm.weight_ih_l0_reverse', 'lstm.weight_hh_l0_reverse', 'lstm.bias_ih_l0_reverse', 'lstm.bias_hh_l0_reverse', 'lstm.weight_ih_l1', 'lstm.weight_hh_l1', 'lstm.bias_ih_l1', 'lstm.bias_hh_l1', 'lstm.weight_ih_l1_reverse', 'lstm.weight_hh_l1_reverse', 'lstm.bias_ih_l1_reverse', 'lstm.bias_hh_l1_reverse', 'fc.weight', 'fc.bias'])


In [30]:
#
params2 = model2.named_parameters()
for name, param in params2:
    if(name == "embedding.weight" or name == "fc.weight" or name == "fc.bias"):
        continue
    else:
        param.data = transfer_param_dict[name]

In [31]:
N_EPOCHS = 10

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model2, train_iterator, optimizer, criterion, tag_pad_idx)
    valid_loss, valid_acc = evaluate(model2, valid_iterator, criterion, tag_pad_idx)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 2.571 | Train Acc: 19.98%
	 Val. Loss: 2.261 |  Val. Acc: 37.95%
Epoch: 02 | Epoch Time: 0m 11s
	Train Loss: 1.915 | Train Acc: 49.32%
	 Val. Loss: 1.726 |  Val. Acc: 55.73%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 1.336 | Train Acc: 67.49%
	 Val. Loss: 1.211 |  Val. Acc: 69.64%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.860 | Train Acc: 81.62%
	 Val. Loss: 0.820 |  Val. Acc: 82.57%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.538 | Train Acc: 90.07%
	 Val. Loss: 0.582 |  Val. Acc: 87.99%
Epoch: 06 | Epoch Time: 0m 11s
	Train Loss: 0.345 | Train Acc: 93.40%
	 Val. Loss: 0.460 |  Val. Acc: 89.54%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.237 | Train Acc: 95.28%
	 Val. Loss: 0.394 |  Val. Acc: 90.71%
Epoch: 08 | Epoch Time: 0m 11s
	Train Loss: 0.173 | Train Acc: 96.61%
	 Val. Loss: 0.353 |  Val. Acc: 91.21%
Epoch: 09 | Epoch Time: 0m 12s
	Train Loss: 0.133 | Train Acc: 97.73%
	 Val. Loss: 0.334 |  Val. Acc: 91.43%
Epoch: 10 | Epoch T

In [32]:
test_loss, test_acc = evaluate(model2, test_iterator, criterion, tag_pad_idx)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.301 |  Test Acc: 92.56%
